# Introduction

## Motivation

## Data


Minsu

# (Adaptive) Kalman Filter

The model assumes there are some latent variables of interest and our observations are linear combinations of them. Let x be the latent states and y be the observed values. Then the model is succinctly stated as

$$x_{t+1} = A x_{t} + u_t$$

$$y_{t+1} = B x_{t+1} + v_t$$

$$u \sim MVN(0, \Sigma_p)$$

$$v \sim MVN(0, \Sigma_m)$$

Under these assumptions, the Kalman Filter is the optimal estimator. We will see why once we connect it to Bayesian statistics. When the $u_t$ and $v_t$ are non-Gaussian, then only approximations exist, often through the Unscented Kalman Filter or Particle Filtering (also known as Sequential Monte Carlo). We will limit ourselves to the Gaussian assumption.

Before we go in depth, we develop the intuition.

## What is a Filter?

A filter only sees past and current data and does not know of the future, but we must make the best guess given our current information. Contrast this with smoothing, which uses future information to make inferences about the present. If we use moving averages, then a sliding window from $t-2$ to $t$ is a filter, while a sliding window from $t-2$ to $t+2$ is a smoother. Visually, a filter looks similar to a smoother. In this illustration, the blue line is the observed values, while the red line is the filtered values.

<img src="gbpusd_example.jpeg" width = "500" />

## Continuous State Space Model

Many people are familiar with Hidden Markov Models.

<img src="HMM.png" width = "500" />

__[<center>Image source</center>](http://www.davidsbatista.net/blog/2017/11/11/HHM_and_Naive_Bayes/)__

A Kalman Filter is like the continuous analog of the HMM. The first equation governs the transition of the hidden states, while the second equation governs the emissions.

The Kalman Filter is used for GPS navigation. Your GPS sends signals (measurements) but they are noisy and the system has to guess where your car actually is. The continuity has two important implications:

* Your car's location isn't restricted to only a few possible states; it can be anywhere on the globe.
* Your car cannot teleport. If for the past hour the measurement comes from New York and the next minute the signal comes from California, we want to keep our estimate close to New York.

Likewise, for our currency model, we don't allow for our currency values to teleport. When reading this report, imagine that the currency intrinsic values are vehicles and the observed exchange rates are GPS readings.

## Connection to Normal-Normal Conjugacy

Consider the simplest case of a Kalman Filter: tracking the position of a point on the Real line. Assume that the point moves according to a random walk with increments $\mathcal N(0, \sigma^2_p)$ and our measurements have an error $\sim \mathcal N(0, \sigma^2_m)$. The prediction variance $s^2_t$ determines the width of our prediction interval; it combines our uncertainty of the location with the uncertainty of the measurement error.

Then the Kalman Filter reduces to (almost) a Normal-Normal conjugacy relationship with unknown mean:

$$\mu_{t+1} = \frac{\frac{1}{\sigma^2_p} \mu_t + \frac{1}{s^2_t} y}{\frac{1}{\sigma^2_p} + \frac{1}{s^2_t}}$$  

$$s^2_{t+1} = \frac{1}{\frac{1}{\sigma^2_p} + \frac{1}{s^2_t}} + \sigma^2_m$$

And here we see why the Kalman Filter is the optimal estimator. Given a prior, the update results in the exact Gaussian posterior, and the estimated states are both the mean and the mode of the posterior. We cannot do any better.

Note the role of $\sigma^2_m$ in this system. We are not assuming a single constant $\mu$. If we did, the posterior variance would get smaller and smaller as we collect more data. However, our data is time-indexed and we at least assume a first-order Markov relationship: the mean of the predictive distribution depends on the unknown location at the current time step. In other words, the $\mu$ is constant and ever-changing.

Here is a short demonstration:


## Parameter Estimation

In virtually all real-world cases, the variances are unknown hyperparameters. The performance of the filter, like most machine learning models, depends heavily on the selected hyperparameters.

Astute readers might note that, in the univariate case, the only thing that matters is the ratio $\frac{\sigma^2_m}{\sigma^2_p}$ or its reciprocal. Likewise, for the multivariate case, the simplest model is to set $\Sigma_m = I_m$ and $\Sigma_p = \tau I_p, \tau \in \mathbb R^+$ and the problem reduces to estimating a single parameter. The next step up in complexity is to use the mean-field variational family: diagonal covariance matrices where we estimate each diagonal element separately.

However, a priori, this is too simple. We expect some currencies to be more stable than others, and exchange rates with the same underlying currency should be correlated with each other. In ADVI, it is noted that financial assets have enough complexity that a full covariance matrix tends to perform much better than the mean-field variational family. Therefore, we want to estimate full-rank matrices.

## Adaptivity

Currency poses another challenge: heteroskedasticity. Many financial time series exhibit volatility clustering, i.e. there are periods of high volatility and periods of low volatility. For this reason, volatility models like GARCH are popular in finance. We do not think the variances stay fixed over all time periods. We want to let it vary over time.

$\Sigma_p$ might increase because of changes in government policy. For instance, an increase in interest rate can cause a currency's intrinsic value to go up.

$\Sigma_m$ might increase in times of uncertainty. For instance, when market participants are uncertain about whether or not a policy will go through, everyone has wildly different guesses of what the "correct" exchange rates should be.

# Challenges

## Markov Assumption

## Covariance Matrices

## Dynamic Systems


Minsu

# Model Fitting

The pseudocode of the algorithm:

```python
initialize priors with kalman_smoother() on burn-in samples
counter = 1
do:
    iter = 0
    while psis_k < 0.7:
        iter = iter + 1
        forward_pass(data[counter:])
    kalman_smoother(data[counter:(counter + iter)])
    update hyperparameters
    counter = counter + iter
until went through all observations
```

To make the problem tractable, we use the inverse Wishart distribution, which is the distribution of covariance matrices characterized by the $\nu$ degrees of freedom parameter and the $S$ scale parameter. Intuitively, $\nu$ is the number of observations we pad our prior with, and $S$ is $X^T X$ where $X$ is a $n \times p$ matrix where each row comes from a $MVN(0, \Sigma)$ distribution. Thus, the update is

$$\nu_{t+1} = \nu_t + n$$

$$S_{t+1} = S_t + S_{obs}$$

We didn't really use Pyro so we're going to smile and say we manually implemented CAVI. Because conjugate priors. :) No questions please. It's probabilistic. And has programming.

## Forward Pass

The forward pass estimates the latent variables by supplying a starting point, some observations, and the covariance matrices. We hold hyperparameters fixed and propagate through the system. I like Wikipedia's version, and will simplify it because we predict $\hat x_{t+1} = x_t$, i.e. $A$ is the identity matrix. $P_{prior}$ is the covariance of the latent states and $e_t$ is the vector of residuals. A single iteration of the filter:

$$P_{pred} = B P_{prior} B^T + \Sigma_p$$

$$e_t = y_t - B \hat x_t$$

$$W_t = B P_{pred} B^T + \Sigma_m$$

$$K_t = P_{pred} B^T W_t^{-1}$$

$$\hat x_t = \hat x_{t-1} + K_t e_t$$

$$P_{post} = (I - K_t B) P_{pred}$$

This looks daunting but we are simply updating our MVN prior with MVN likelihood and then adding some uncertainty at each time step.

## EM Smoothing

The Rauch-Tung-Striebel (forward-backward) algorithm is the most common form of Kalman Smoothing. However, even this algorithm requires known covariance matrices. Also, RTS provides the best guess of the initial state, but if we do that, then we allow for jump discontinuities. As stated before, we do not think the intrinsic values can teleport.

Instead, we hold the initial state constant, and we get the maximum likelihood estimate of the covariance matrix. In the expectation step, we do a forward_pass() to estimate the latent states given the covariances. In the maximization step, we do

$$\hat S_m = (Y-\hat Y)^T (Y-\hat Y)$$

$$\hat S_p = \sum_t (X_t - X_{t-1})^T (X_t - X_{t-1})$$

We iterate EM until convergence. Because of numerical stability issues (to be discussed later) the criterion for convergence is rather loose.

We then update our inverse Wishart priors using the empirical scale matrices. The updates to degrees of freedom is obvious.


## Pareto-Smoothed-Importance-Sampling Leave-Future-Out Cross-Validation (PSIS-LFO-CV)

Wow, what a mouthful.

PSIS is often used to approximate LOO-CV to assess model fit. The approximation can become really bad if the raw importance weights have infinite variance. We don't want LOO-CV because it is a form of smoothing--future data contaminates current estimates. LFO-CV is a form of CV for filtering problems, and the raw importance weights are proportional to

$$r_n^{(s)} \propto \prod_{i=1}^n p(y_{t+i}|\theta^{(s)})$$

Which is the likelihood of the observations given some sampled hyperparameters. In other words, we can sample some covariance matrices and hold them fixed, and given an initial state and some observations, we can go through the forward_pass() and evaluate the likelihood of the observations up to that time point. When the likelihoods start to have infinite variance, our model is becoming unstable because their predictions start to differ wildly.

PSIS fits a generalized Pareto distribution to the right tail of the raw importance weights (the highest 20%) with some weakly regularizing prior (because otherwise fitting a GPD can be extremely hard). When the shape parameter $k$ goes above 0.5, the variance of the distribution is infinite. However, in the PSIS-LOO-CV paper, empirically the results are decent until $k > 0.7$, at which point we update our hyperparameters and start a new forward_pass().


# Results

## Predictive Performance

## Latent State Estimates



# Criticism

## Computational Complexity

## Serially Correlated Errors

## Acceleration Term

Minsu

# Conclusion

Wicak

# References

__[ADVI](https://arxiv.org/pdf/1603.00788.pdf)__ Kucukelbir et al. (2016). Automatic Differentiation Variational Inference

__[PSIS](https://arxiv.org/pdf/1507.02646.pdf)__ Vehtari et al. (2019). Pareto Smoothed Importance Sampling

__[PSIS-LOO-CV](https://arxiv.org/pdf/1507.04544.pdf)__ Vehtari et al. (2016) Practical Bayesian model evaluation using leave-one-out cross-validation and WAIC

__[PSIS-LFO-CV](https://arxiv.org/pdf/1902.06281.pdf)__ Bürkner et al. (2019). Approximate leave-future-out cross-validation for Bayesian time series models

